# Exercise 3

Each exercise (except for the first) corresponds to one python file. The results can be replicated by running these code files directly (setup and configuration is done in the `main` section at the bottom of the files) or the notebook.

## 1. Bayes' Theorem

Note: We assume that $0.5%$ of the population are drug users, not $0.05%$ - otherwise the percentages of drug users and non-users don't add up to 100.
No code is provided for this example, results were computed manually.

$P(A|B) = P(B|A)P(A) / P(B)$

Here: 
* $P(A|B)$ -> Probability of observing that a randomly selected individual is not a drug user given a positive drug test. $P(A|B)$ = ?
* $A$ -> Not a drug user. $P(A) = 0.995$, $P(¬A) = 1 - P(A) = 0.005$.
* $B$ -> Positive drug test. $P(B) = P(A) * 0.01 + P(¬A) * 0.99 = 0.995 * 0.01 + 0.005 * 0.99 = 0.00995 + 0.00495 = 0.0149$.
* $P(B|A)$ -> Probability of observing a positive drug test given a non-user. $P(B|A) = 0.01$.

Hence:
$P(A|B) = 0.01 * 0.995 / 0.0149 = 0.6677852349 ~ 66.7%$

**Interpretation**: Due to the group of non-users being the vast majority, the test (with a sensitivity of 99%) generates more false positives than true positives. 

## 2. A-B Testing

Code can be found in `a_b_testing.py`. 

In [3]:
import pandas as pd
from scipy.stats import chi2_contingency, binom_test
from scipy.special import ndtr
import numpy as np

# Data pre-processing.
df = pd.read_csv("ab_data.csv")
num_new_page = len(df[df.landing_page == "new_page"])
num_old_page = len(df[df.landing_page == "old_page"])
num_conversions = len(df[df.converted == 1])
num_conversions_old = len(df[(df.landing_page == "old_page") & (df.converted == 1)])
num_conversions_new = len(df[(df.landing_page == "new_page") & (df.converted == 1)])
contingency_table = pd.crosstab(df.landing_page, df.converted)

### Traffic Split and Conversion Rate
*(a) In what proportion did the company split the landing page?*  
40000:60000 = 40.0% of all visits were redirected to the new page.   

*(b) What is the conversion rate for each version of the landing page?*  
* Old page: $0.12063333333333333$. 
* New page: $0.1191$.

In [4]:
    # 2.1a. In what proportion did the company split the landing page?
    print(
        "2.1a. In what proportion did the company split the landing page?\n",
        str(num_new_page) + ":" + str(num_old_page),
        "= " + str(num_new_page / len(df) * 100) + "% of all visits were redirected to the new page. "
    )

    # 2.1b. What is the conversion rate for each version of the landing page?
    print(
        "2.1b. What is the conversion rate for each version of the landing page?\n",
        " Old page: " + str(num_conversions_old / num_old_page) + ".",
        "\n  New page: " + str(num_conversions_new / num_new_page) + ".",
    )

2.1a. In what proportion did the company split the landing page?
 40000:60000 = 40.0% of all visits were redirected to the new page. 
2.1b. What is the conversion rate for each version of the landing page?
  Old page: 0.12063333333333333. 
  New page: 0.1191.


### Binomial Test 

We phrase $H_0$ as follows: The probability of observing the specified number of conversions with the specified landing page version is not higher than 50%. This is a rephrased variant of the original hypothesis of "the probability of conversion does not depend on the landing page". We used $p = 0.5$ to reflect the equaliy assumption in $H_0$.

Probability of observing observed number of conversations given the assumptions above and using scipy's binomial test:
* Old page: $p = 6.317471224286417e-114$. 
* New page: $p = 0.9999999999999999$.

Assuming a significance level of $0.05$, we can interpret this result as follows: 
* The probability for the old landing page is lower than our alpha. This means we can reject $H_0$ - 
that the old landing page's chance of success is not higher than 50% when compare with the new landing page.
* The probability for the new landin page is higher than our alpha. This means we cannot reject $H_0$ and the new landing page's chance of success compared to the old version is indeed not higher than 50%.

Intuitively speaking, this makes sense - the number of conversions stemming from the old landing page are considerably 
higher than the one from the new one and we have thousands of samples, so we would expect the old landing page having a 
more than even chance of success when compared to the new version.
  


In [6]:
# 2.2. Under the hypothesis that the probability of conversion does not depend on the version of the landing page
# with the help of a binomial-test find how likely it is to observe the number of conversions as extreme as the one
# for the old landing page and the one for the new landing page.
print(
    "\n2.2. Probability of observing observed number of conversations if probability of conversation does not "
    "depend on landing page version.\n",
    " Old page: " + str(binom_test(num_conversions_old, n=num_conversions, p=0.5, alternative='greater')) + ".",
    "\n  New page: " + str(binom_test(num_conversions_new, n=num_conversions, p=0.5, alternative='greater')) + "."
)


2.2. Probability of observing observed number of conversations if probability of conversation does not depend on landing page version.
  Old page: 6.317471224286417e-114. 
  New page: 0.9999999999999999.


### Chi-squared and Normal Test

We assume $H_0$ to state that there is no significant difference in conversion rates between landing page versions.

Significances in differences in conversion rates w.r.t. landing page versions.
* χ2: $p = 0.4709069331806466$.
* normality test: $p = 0.6616312516159468$.

The χ2 result was computed by aggregating the available data to a contingency table before using scipy's 
`chi2_contingency`. The normal test was computed manually by following the equation in the slides - computing 
$t_a, t_b, z$ as consequence of the number of successes and the numbers of tries and then retrieving the p-value via the computed z-score.

We can interpret both (χ2 and normal test) results the same way: We can hence reject $H_0$, i. e. the claim that there is significant differences in conversion rates between landing pages is false.
This conclusion is in agreement with both the result from the binomial tests and our intuition.

In [7]:
# 2.3. Under the same null hypothesis use the χ-squared test and the normal-test to measure the significance of the
# difference in the conversion-rates of the landing page versions. Same H_0 as in 2.2.

t_a = num_conversions_old / num_old_page
t_b = num_conversions_new / num_new_page
z = (t_a - t_b) / np.sqrt(
    np.std(df[df.landing_page == "old_page"].converted) / num_old_page +
    np.std(df[df.landing_page == "new_page"].converted) / num_new_page
)

stat, p_chi2, dof, expected = chi2_contingency(contingency_table.values)
print(
    "\n2.3. Significances in differences in conversion rates w.r.t. landing page versions.\n",
    "χ2: p = " + str(p_chi2) +
    "\n normality test: p = " + str(ndtr(z))
)


2.3. Significances in differences in conversion rates w.r.t. landing page versions.
 χ2: p = 0.4709069331806466
 normality test: p = 0.6616312516159468


## 3. Logistic Regression

Code can be found in `logistic_regression.py`.

We compared our implementation with sklearn's `sklearn.linear_model.LogisticRegression` and mimicked parts of its interfaces. While our model takes considerably longer to converge, it approximates sklearn's accuracy closely when the right learning rate is chosen. We choose the F1 score to reflect the quality of the prediction for our evaluation. Our implementation stops whenver the maximum number of iterations is reached or the result of the loss function converges to a value that is lower than the specified threshold. We used a train-test split with a proportion of $\frac{2}{3}$ : $\frac{1}{3}$.

Utilizing a grid search, the best learning rate seems to be 0.001. In the following a list of tried learning rate values 
together with the computed F1-score:
```
  0.05:    0.6011731155616048
  0.01:    0.5804276660764605
  0.005:   0.6407631173854077
  0.001:   0.7605988196913313
  0.0005:  0.7558456670258468
  0.0001:  0.7300973641595375
  sklearn: 0.7616874084952325
```
As can be seen here, sklearn's implementation beats our vanilla implementation (using a learning rate of $0.001$) by a tiny fraction, but the difference is marginal. Note that we didn't optimize the hyperparameters for sklearn's LR however.

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as SKLearnLogisticRegression
from typing import Tuple
from logistic_regression import LogisticRegression, evaluate


wine_df = pd.read_csv("winequality_binary.csv").drop(columns=["Unnamed: 0"])
features = wine_df.drop(columns=["quality"])
labels = wine_df[["quality"]]

# Check distribution of class labels.
unique, counts = np.unique(labels.values, return_counts=True)
print(counts)

#############################################################
# 3. Logistic regression.
#############################################################

print("*** Evalution + hyperparameter search ***")

# Search for best parameter for learning rate, compare with sklearn implementation.
learning_rates = (5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4)
n_splits = 3
f1_scores = {lr: 0 for lr in learning_rates}
f1_scores["sklearn"] = 0

for j in range(n_splits):
    print("  In split", j + 1)

    X_train, X_test, y_train, y_test = train_test_split(features.values, labels.values, test_size=0.33)

    # Evaluate own implementation with different parametrizations.
    for learning_rate in learning_rates:
        print("    learning rate =", learning_rate)
        tp, fp, tn, fn = evaluate(X_train, y_train, X_test, y_test, learning_rate=learning_rate)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1_scores[learning_rate] += 2 * precision * recall / (precision + recall)

    # Evaluate with sklearn.
    tp, fp, tn, fn = evaluate(X_train, y_train, X_test, y_test, use_own=False)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_scores["sklearn"] += 2 * precision * recall / (precision + recall)

print()
for config in f1_scores:
    f1_scores[config] /= n_splits
    print("  " + str(config) + ": " + str(f1_scores[config]))

[744 855]
*** Evalution + hyperparameter search ***
  In split 1
    learning rate = 0.05


/home/raphael/Development/univie-business-intelligence/exercises/2/logistic_regression.py:70: RuntimeWarning: divide by zero encountered in log
  cost = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
/home/raphael/Development/univie-business-intelligence/exercises/2/logistic_regression.py:70: RuntimeWarning: invalid value encountered in multiply
  cost = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()


    learning rate = 0.01
    learning rate = 0.005
    learning rate = 0.001
    learning rate = 0.0005
    learning rate = 0.0001


/home/raphael/.conda/envs/univie-busint/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raphael/Development/univie-business-intelligence/exercises/2/logistic_regression.py:70: RuntimeWarning: divide by zero encountered in log
  cost = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
/home/raphael/Development/univie-business-intelligence/exercises/2/logistic_regression.py:70: RuntimeWarning: invalid value encountered in multiply
  cost = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()


  In split 2
    learning rate = 0.05
    learning rate = 0.01
    learning rate = 0.005
    learning rate = 0.001
    learning rate = 0.0005
    learning rate = 0.0001


/home/raphael/.conda/envs/univie-busint/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/raphael/Development/univie-business-intelligence/exercises/2/logistic_regression.py:70: RuntimeWarning: divide by zero encountered in log
  cost = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
/home/raphael/Development/univie-business-intelligence/exercises/2/logistic_regression.py:70: RuntimeWarning: invalid value encountered in multiply
  cost = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()


  In split 3
    learning rate = 0.05
    learning rate = 0.01
    learning rate = 0.005
    learning rate = 0.001
    learning rate = 0.0005
    learning rate = 0.0001

  0.05: 0.6011731155616048
  0.01: 0.5804276660764605
  0.005: 0.6407631173854077
  0.001: 0.7605988196913313
  0.0005: 0.7558456670258468
  0.0001: 0.7300973641595375
  sklearn: 0.7616874084952325


/home/raphael/.conda/envs/univie-busint/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## 4. Confidence Interval of the Prediction Accuracy

We assumed a confidence level of $0.95$ and ran all tests with our vanilla implementatin of logistic regresion and the 
optimal learning rate found in the previous task. We implemented pseudocode for bootstrapping we found online and the
same evaluation methds as for the previous task to gather a accuracy statistic. Following that, we pick the alpha-/
confidence level-percentiles from this statistic.

We used a train-test split with a proportion of $\frac{1}{2}$:$\frac{1}{2}$, following sklearn's default values. We ran 50 iterations. 
Predicting the results with these pre-conditions yields a $0.95$-confidence of $(69.42187499999999, 75.27604166666666)$.

In [12]:
from bootstrapping import compute_bootstrap_accuracy

wine_df = pd.read_csv("winequality_binary.csv").drop(columns=["Unnamed: 0"])
features = wine_df.drop(columns=["quality"])
labels = wine_df[["quality"]]

#############################################################
# Confidence interval of prediction accuracy.
#############################################################

print("*** Bootstrapping ***")
print(compute_bootstrap_accuracy(features.values, labels.values, n_bootstraps=50, n_train=0.7))


*** Bootstrapping ***
  Iteration #0
0.7229166666666667
  Iteration #1
0.7208333333333333
  Iteration #2
0.73125
  Iteration #3
0.7145833333333333
  Iteration #4
0.7166666666666667
  Iteration #5
0.7166666666666667
  Iteration #6
0.7166666666666667
  Iteration #7
0.7333333333333333
  Iteration #8
0.71875
  Iteration #9
0.7270833333333333
  Iteration #10
0.7020833333333333
  Iteration #11
0.71875
  Iteration #12
0.7375
  Iteration #13
0.7041666666666667
  Iteration #14
0.7208333333333333
  Iteration #15
0.7375
  Iteration #16
0.6958333333333333
  Iteration #17
0.7333333333333333
  Iteration #18
0.725
  Iteration #19
0.69375
  Iteration #20
0.7125
  Iteration #21
0.7354166666666667
  Iteration #22
0.6895833333333333
  Iteration #23
0.70625
  Iteration #24
0.7291666666666666
  Iteration #25
0.7020833333333333
  Iteration #26
0.7229166666666667
  Iteration #27
0.73125
  Iteration #28
0.7145833333333333
  Iteration #29
0.73125
  Iteration #30
0.7166666666666667
  Iteration #31
0.74166666666